In [2]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

/home/paperspace/anaconda3/envs/py2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:556: UserWarning: Theano flag device=gpu* (old gpu back-end) only support floatX=float32. You have floatX=float64. Use the new gpu back-end with device=cuda* for that value of floatX.
  warnings.warn(msg)
Using gpu device 0: Quadro P5000 (CNMeM is disabled, cuDNN 6021)
/home/paperspace/anaconda3/envs/py2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:631: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn(warn)


In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [4]:
path = get_file('lorem-legal.txt', origin="https://drive.google.com/uc?authuser=0&id=0B9vWSUR3kTPfcEMwMlhYRWNjYWs&export=download")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 7813987


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 88


In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n "$%&\'()*,-./0123456789:;<>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ`abcdefghijklmnopqrstuvwx'

In [8]:
''.join(chars)

'\x00\n "$%&\'()*,-./0123456789:;<>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ`abcdefghijklmnopqrstuvwxyz\x80\x90\x94\xe2'

In [9]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [10]:
idx = [char_indices[c] for c in text]

In [11]:
idx[:10]

[46, 62, 75, 70, 66, 76, 76, 66, 72, 71]

In [12]:
''.join(indices_char[i] for i in idx[:70])

'Permissions for Redistribution of the Work, you may use, reproduce, di'

In [13]:
cs=3
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [14]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])
y = np.stack(c4_dat[:-2])

In [ ]:
dense_input = Dense(256, activation='relu')

In [ ]:
len(chars)

In [ ]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int32', name=name)
    emb = Embedding(n_in, n_out)(inp)
    return inp, Flatten()(emb)

In [ ]:
latent_factors = 42
c1_in, c1 = embedding_input('c1_input', len(chars), latent_factors)
c2_in, c2 = embedding_input('c2_input', len(chars), latent_factors)
c3_in, c3 = embedding_input('c3_input', len(chars), latent_factors)

In [ ]:
n_hidden = 256
dense_in = Dense(output_dim=n_hidden, activation='relu')

In [ ]:
dense_h = Dense(output_dim=n_hidden, activation='tanh')

In [ ]:
c1_h = dense_in(c1)

In [ ]:
c2_h = dense_in(c2)
c2_hc1 = dense_h(c1_h)
c2_m = merge([c2_h, c2_hc1])

In [ ]:
c3_h = dense_in(c3)
c3_hc2 = dense_h(c2_m)
c3_m = merge([c3_h, c3_hc2], mode='concat')

In [ ]:
dense_out = Dense(output_dim=len(chars), activation='softmax')

In [ ]:
c4_out = dense_out(c3_m)

In [ ]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
model.optimizer.lr = 0.00001

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

## Creating first RNN Model

In [23]:
cs=8

In [24]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

In [25]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [26]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [27]:
len(xs), xs[0].shape

(8, (976746,))

In [28]:
y = np.stack(c_out_dat[:-2])

In [29]:
[xs[n][:cs] for n in range(cs)]

[array([46, 72, 48, 59, 63, 75, 70,  2]),
 array([62, 71, 62, 78,  2, 68, 58, 75]),
 array([75, 76, 61, 77, 77, 11, 82, 62]),
 array([70,  2, 66, 66, 65,  2,  2, 73]),
 array([66, 63, 76, 72, 62, 82, 78, 75]),
 array([76, 72, 77, 71,  2, 72, 76, 72]),
 array([76, 75, 75,  2, 53, 78, 62, 61]),
 array([66,  2, 66, 72, 72,  2, 11, 78])]

In [30]:
y[:cs]

array([72, 48, 59, 63, 75, 70,  2, 60])

### Returning sequences

In [35]:
#c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [36]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [37]:
[xs[n][:cs] for n in range(cs)]

[array([46, 72, 48, 59, 63, 75, 70,  2]),
 array([62, 71, 62, 78,  2, 68, 58, 75]),
 array([75, 76, 61, 77, 77, 11, 82, 62]),
 array([70,  2, 66, 66, 65,  2,  2, 73]),
 array([66, 63, 76, 72, 62, 82, 78, 75]),
 array([76, 72, 77, 71,  2, 72, 76, 72]),
 array([76, 75, 75,  2, 53, 78, 62, 61]),
 array([66,  2, 66, 72, 72,  2, 11, 78])]

In [38]:
[ys[n][:cs] for n in range(cs)]

[array([62, 71, 62, 78,  2, 68, 58, 75]),
 array([75, 76, 61, 77, 77, 11, 82, 62]),
 array([70,  2, 66, 66, 65,  2,  2, 73]),
 array([66, 63, 76, 72, 62, 82, 78, 75]),
 array([76, 72, 77, 71,  2, 72, 76, 72]),
 array([76, 75, 75,  2, 53, 78, 62, 61]),
 array([66,  2, 66, 72, 72,  2, 11, 78]),
 array([72, 48, 59, 63, 75, 70,  2, 60])]

## Model with Keras

In [ ]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [ ]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

## Sequence model with Keras

In [17]:
n_hidden = 256
n_fac = 42

In [18]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

In [19]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 8, 256)        76544       embedding_1[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_1[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [31]:
xs[0].shape

(976746,)

In [39]:
x_rnn=np.stack(np.squeeze(xs), axis=1)
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [40]:
x_rnn.shape, y_rnn.shape

((976746, 8), (976746, 8, 1))

In [ ]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

In [42]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [ ]:
get_nexts_keras(' this is')

## Stateful model in Keras

In [43]:
bs=64

In [48]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        GRU(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [49]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [50]:
mx = len(x_rnn)//bs*bs

In [51]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
976704/976704 [==============================] - 246s - loss: nan   
Epoch 2/4
694784/976704 [====================>.........] - ETA: 66s - loss: nan

KeyboardInterrupt: 

In [ ]:
model.optimizer.lr=1e-4

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

In [ ]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)